In [ ]:
#Original code for this cell at https://github.com/theofil/dscout/blob/master/analysis/exampleAnalysis.ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import subprocess
import bz2

plt.rc('font', size=22)

#I used the code in your repository to handle the files, I hope you don't mind.
#I commented out (starting with ##) the lines which I do not know what they do/ which I do not believe I need


# data files
path = './data/hiion/'
files = os.listdir(path)
files = [f for f in files if 'monitor' not in f] #----> You had <if 'monitor' in f> but I believe I need those without 'monitor'

# getrunNumber from string e.g., int('/data/hiion/scout_326676_000000.monitor.txt'.split('_')[1]
getRun = lambda x: int(x.split('_')[1])

# exclude runs
##underflowRuns = [326676, 326883, 327401]
##overflowRuns  = [326434, 326897]
##files = [f for f in files if getRun(f) not in underflowRuns]

# list to hold dataframes
dfs = []
runIndex= []

for file in files:
    filepath = path+file
    df = pd.read_csv(filepath)
    
    # integrity checks
    ##if df.columns[0] != '0':print(f'underflow was found in {filepath}') 
    ##if df.iloc[3564].to_numpy()[0] != 0: print(f'overflow was found in {filepath}')    
    
    # drop overflow row
    ##df = df[:-1]
    
    ## reindex such that index = bx
    ##df.index = pd.RangeIndex(start=1, stop=3565, step=1)
    
    # add a column with the run number
    df['run'] = getRun(file) 
    dfs += [df]
    
    #make in index with the run numbers
    runIndex += [getRun(file)]
plt.rc('figure', max_open_warning = 100)


dfs[15]['run']=dfs[15]['run']+0.000001
runIndex[15]= runIndex[15]+0.000001
##goodBX  = lambda bx: True if bx >= 3445 and  bx <= 3555 else False
##bxMin = 3445-3564
##bxMax = 3555-3564

In [ ]:
phiCounts= []
for df in dfs:
    phiticks = [0.5*np.pi*i for i in range(-2,3)]
    plt.rc('font', size=22)
    plt.figure(figsize=(8,8))
    counts, bins, bars = plt.hist(df['phi'], bins = 30, histtype = 'bar', ec='black')
    phiCounts.append(counts)
    plt.xticks(phiticks)
    plt.title('Run {}'.format(np.split(df['run'].values,[i for i in range(1,len(df)+1)])[0]))
    plt.xlabel('$\phi$ [degrees]')
    plt.ylabel('# muon candidates');
    
phiCounts=np.array(phiCounts)

In [ ]:
orbCounts = []
for df in dfs:
    plt.rc('font', size=22)
    plt.figure(figsize=(8,8))
    counts, bins, bars = plt.hist(df['orbit'], bins=30,  histtype = 'bar', ec='black')
    orbCounts.append(counts)
    plt.title('Run {}'.format(np.split(df['run'].values,[i for i in range(1,len(df)+1)])[0]))
    plt.xlabel('orbit')
    plt.ylabel('# muon candidates');

orbCounts=np.array(orbCounts)

We can see that certain $\phi$ graphs have ominus spikes at 0 and 180 degrees. Some graphs don't even have the two gaussian-like shapes that are observed in most of the graphs.
Also regarding the orbit graphs, we can see in a lot of them large spikes and gaps. 
Some (certainly not all) of these behaviors are due to small amount of data in some runs, but for the ones that have a good amount of data but do not comply with the consant-like shape we expected, should we look at what was going on with the detector at that point? 

Next, I want to see how the orbit numbers match the transition from one run to another.

In [ ]:
print(dfs[0]['orbit'].min()==dfs[0]['orbit'].values[0])
print(dfs[0]['orbit'].max()==dfs[0]['orbit'].values[-1])
#min and max match first and last orbit numbers

In [ ]:
orbitLims=np.zeros((len(dfs),7))
for i in range(len(dfs)):
    orbitLims[i][0] = dfs[i]['orbit'].min()
    orbitLims[i][1] = dfs[i]['orbit'].max()
    orbitLims[i][2] = dfs[i]['orbit'].max()-dfs[i]['orbit'].min()
    orbitLims[i][3] = dfs[i]['bx'].min()
    orbitLims[i][4] = dfs[i]['bx'].max()
    orbitLims[i][5] = dfs[i]['bx'].max()-dfs[i]['bx'].min()
    orbitLims[i][6] = len(dfs[i])

In [ ]:
odf = pd.DataFrame(orbitLims, index=runIndex, columns='orbit_min orbit_max orbit_range bx_min bx_max bx_range No_of_events'.split())
odf # orbit data frame

In [ ]:
odf.describe()
#why is the count of everthing except No_of_events 81 instead of 84?

It seems like the orbit count resets with each run.
We should make a universal clock.
Also, the bxs included in each file seem to be corect, ignoring the fact that  max(bx_min)=3447 and min(bx_max)=3550,  so we should look at what causes the spikes at $\phi=0 $ or $\pi$. You mentioned some kind of collision contamination in the abort gap. The only way I can think to check this statement is by compairing the number of events in a problematic run to the avg value of events in all the "good" runs, but maybe this method introduces unwanted biases (= I assume which runs are "good" with my common sense which may be wrong). 